In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
!pip install emoji
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
import emoji
nltk.download('stopwords')
from collections import Counter
from nltk.tokenize import word_tokenize
!pip install fasttext
import fasttext
import ast
import json
import fasttext.util

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 9.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.1-py3-none-any.whl (238 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246762 sha256=feed0004fe45b6aa38a997ecfc02649cbde8e5c1c09b36de239baba4c519d8b9
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Internship_NIT_Patna/binary_combined_data.csv")

In [ ]:
data

,tweet_id,text,binary_label
0,1214794148504457217,It’s not a fight for dharma it’s embracing it....,NOT
1,1242560043494621185,bhenchod mulk ki maa chud rahi hai aur tum log...,HOF
2,1335539265015517186,#babrimasjid\n#BabriMasjid was and always will...,NOT
3,1467692528308686850,#babrimasjid\n#BabriMasjid was and always will...,HOF
4,1467694052644511745,#babrimasjid\n#BabriMasjid was and always will...,HOF
...,...,...,...
4909,1534729188842033152,One woman 57 muslim country 😂😂 #QatarExposed h...,NOT
4910,1534750107673010176,One woman 57 muslim country 😂😂 #QatarExposed h...,HOF
4911,1534754537914806272,One woman 57 muslim country 😂😂 #QatarExposed h...,HOF
4912,1534757926853783552,One woman 57 muslim country 😂😂 #QatarExposed h...,HOF


In [ ]:
def load_and_preprocess_data(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    print(df)
    print(df['binary_label'].value_counts())

    return df

In [ ]:
file_path= r'/content/drive/MyDrive/Internship_NIT_Patna/binary_combined_data.csv'

In [ ]:
# Load and preprocess the data
df = load_and_preprocess_data(file_path)

                 tweet_id                                               text  \
0     1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1     1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2     1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3     1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4     1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   
...                   ...                                                ...   
4909  1534729188842033152  One woman 57 muslim country 😂😂 #QatarExposed h...   
4910  1534750107673010176  One woman 57 muslim country 😂😂 #QatarExposed h...   
4911  1534754537914806272  One woman 57 muslim country 😂😂 #QatarExposed h...   
4912  1534757926853783552  One woman 57 muslim country 😂😂 #QatarExposed h...   
4913  1534722278768136192  One woman 57 muslim country 😂😂 #QatarExposed h...   

     binary_label  
0             NOT  

In [ ]:
def clean_text(text):
    # Convert emojis to text
#     text = emoji.demojize(text, delimiters=("", ""))
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove digits
    text = re.sub(r'\d+', '', text)

    # Remove special characters except for spaces and Hindi characters
    text = re.sub(r'[^\w\s\u0900-\u097F]', '', text)  # Keep Hindi characters and spaces

    # Normalize multiple spaces to a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def load_data(file_path):
    df = pd.read_csv(file_path)

    # Apply text cleaning
    df['cleaned_text'] = df['text'].apply(clean_text)

    # Drop the original 'text' column if no longer needed
    df.drop(['text'], axis=1, inplace=True)

    return df
# Load and preprocess the data
data = load_data(file_path)
print(df.head())

              tweet_id                                               text  \
0  1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1  1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2  1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3  1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4  1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   

  binary_label  
0          NOT  
1          HOF  
2          NOT  
3          HOF  
4          HOF  


In [ ]:
df['cleaned_text'] = df['text'].apply(lambda text: clean_text(text) if pd.notnull(text) else text)

In [ ]:
nltk_stopwords = set(stopwords.words('english'))

In [ ]:
text_column = 'text'

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def extract_stopwords(text):
    words = re.findall(r'\b\w+\b', text.lower())
    stopwords_in_text = [word for word in words if word in stop_words]
    return stopwords_in_text

In [ ]:
all_stopwords = []
for text in df[text_column].dropna():
    all_stopwords.extend(extract_stopwords(text))

In [ ]:
all_stopwords

['it',
 's',
 'not',
 'a',
 'for',
 'it',
 's',
 'it',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'are',
 'do',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 'are',
 'a',
 'the',
 'will',
 'on',
 'which',
 'the',
 'of',
 'will',
 'be',
 'in',
 'the',
 't',
 'was',
 'and',
 'will',
 'be',
 'a',
 'and',
 '

In [ ]:
vocabulary_of_stopwords = set(all_stopwords)

In [ ]:
print("Vocabulary of stopwords:")
print(vocabulary_of_stopwords)

Vocabulary of stopwords:
{'and', 'yours', 'weren', 'nor', 'you', 'if', 'we', 'does', 's', 'having', 'out', 'has', 'most', 'them', 'his', 'further', 'theirs', 'just', 'me', 'between', 'd', 'herself', 'before', 'same', 'didn', 'have', 'him', 'the', 'won', 'such', 'did', 'itself', 'y', 't', 'o', 'into', 'shouldn', 'a', 'these', 'they', 'from', 'about', 'by', 'again', 'been', 'until', 'will', 'each', 'any', 'as', 'isn', 'its', 'under', 'it', 'too', 'few', 'not', 'don', 'hasn', 'my', 'be', 'during', 'to', 'very', 'haven', 'other', 'once', 'can', 'couldn', 'm', 'now', 'wouldn', 'here', 'those', 'aren', 'but', 'he', 're', 'hers', 'more', 'yourself', 'himself', 'their', 'was', 'both', 'doing', 'should', 'where', 'below', 'of', 'do', 'or', 'what', 'ourselves', 'were', 'while', 'after', 'ours', 'this', 'how', 'hadn', 'being', 'some', 'her', 'off', 'at', 'i', 'who', 'against', 'so', 'up', 'because', 'your', 'is', 'our', 'themselves', 'she', 'down', 'for', 'only', 'yourselves', 'then', 'with', 'mu

In [ ]:
stopword_counts = Counter(all_stopwords)
print("\nMost common stopwords:")
print(stopword_counts.most_common())


Most common stopwords:
[('the', 6927), ('to', 4581), ('t', 4180), ('in', 3815), ('and', 3795), ('is', 3793), ('of', 3763), ('a', 3229), ('for', 2842), ('s', 2293), ('you', 2216), ('was', 1809), ('on', 1761), ('it', 1730), ('this', 1520), ('by', 1454), ('be', 1346), ('are', 1308), ('that', 1304), ('who', 1260), ('not', 1228), ('have', 1144), ('i', 1090), ('if', 1084), ('no', 1076), ('from', 1002), ('they', 973), ('will', 969), ('she', 890), ('with', 855), ('can', 834), ('we', 814), ('as', 794), ('what', 775), ('at', 667), ('so', 666), ('her', 655), ('has', 648), ('he', 637), ('its', 636), ('your', 619), ('but', 601), ('when', 594), ('their', 578), ('such', 549), ('against', 541), ('why', 539), ('just', 526), ('own', 520), ('an', 480), ('about', 476), ('itself', 470), ('or', 469), ('there', 464), ('all', 446), ('were', 438), ('his', 421), ('do', 412), ('me', 405), ('any', 384), ('don', 360), ('only', 348), ('them', 348), ('how', 337), ('him', 325), ('after', 319), ('here', 315), ('more'

In [ ]:
def remove_english_stopwords(text):
    words = nltk.word_tokenize(text.lower())
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
# Apply stopword removal to the dataset
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: remove_english_stopwords(text) if pd.notnull(text) else text)

In [ ]:
hindi_stopwords = [
    "अंदर", "अगर", "अत", "अथवा", "अदि", "अधिक", "अधिकांश", "अन्य", "अप", "अपना", "अपनि", "अपनी", "अपने", "अब", "अभि", "अभी", "आगे", "आज", "आदि", "आप", "आपका",
    "इंहिं", "इंहें", "इंहों", "इतयादि", "इत्यादि", "इन", "इनका", "इनकी", "इनके", "इन्हीं", "इन्हें", "इन्हों", "इस", "इसका", "इसकि", "इसकी", "इसके", "इसमें",
    "इसलिए", "इसि", "इसी", "इसे", "उंहिं", "उंहें", "उंहों", "उन", "उनका", "उनकि", "उनकी", "उनके", "उनको", "उन्हीं", "उन्हें", "उन्हों", "उन्होने", "उस", "उसका",
    "उसकी", "उसके", "उसि", "उसी", "उसे", "ऊपर", "एक", "एवं", "एस", "एसे", "ऐसा", "ऐसे", "ओर", "और", "कइ", "कई", "कब", "कभी", "कम", "कर", "करता", "करते",
    "करना", "करने", "करें", "कल", "कहते", "कहा", "कहाँ", "का", "काफि", "काफ़ी", "कि", "किंहें", "किंहों", "किए", "कितना", "किन्हें", "किन्हों", "किया", "किर",
    "किस", "किसको", "किसि", "किसी", "किसे", "की", "कुछ", "कुल", "के", "कैसे", "को", "कोइ", "कोई", "कोन", "कोनसा", "कौन", "कौनसा", "क्या", "क्यूंकि", "क्यों",
    "खिलाफ", "खुद", "गई", "गए", "गया", "गयी", "गये", "घर", "चाहिए", "जब", "जबकि", "जरा", "जहाँ", "जहां", "जा", "जाता", "जाती", "जाते", "जाने", "जायेंगे",
    "जिंहें", "जिंहों", "जितना", "जिधर", "जिन", "जिन्हें", "जिन्हों", "जिस", "जिसमें", "जिससे", "जिसे", "जीधर", "जेसा", "जेसे", "जैसा", "जैसे", "जो", "ज्यादा",
    "तक", "तथा", "तब", "तरह", "तहत", "तिंहें", "तिंहों", "तिन", "तिन्हें", "तिन्हों", "तिस", "तिसे", "तुम", "तुम्हारा", "तो", "था", "थि", "थी", "थे", "दबारा",
    "दवारा", "दिया", "दुबारा", "दुसरा", "दुसरे", "दूर", "दूसरा", "दूसरे", "दो", "दोनो", "दोनों", "दौरान", "द्वारा", "न", "नहिं", "नहीं", "ना", "निकल", "निचे",
    "निहायत", "नीचे", "ने", "पडा", "पर", "परंतु", "पहले", "पुरा", "पूरा", "पूरे", "पे", "प्रति", "प्रत्येक", "फिर", "बंद", "बड़ा", "बड़े", "बनि", "बनी", "बहि",
    "बही", "बहुत", "बाद", "बार", "बारे", "बाला", "बाहर", "बिलकुल", "बीच", "भि", "भितर", "भी", "भीतर", "मगर", "मात्र", "माध्यम", "मानो", "मुझको", "मे", "में",
    "मेरा", "मै", "मैं", "यदि", "यह", "यहाँ", "यहां", "यहि", "यही", "या", "यिह", "ये", "रखें", "रवासा", "रहती", "रहा", "रही", "रहे", "ऱ्वासा", "लिए", "लिया",
    "लिये", "लेकर", "लेकिन", "व", "वगेरह", "वरग", "वर्ग", "वह", "वहाँ", "वहां", "वहिं", "वहीं", "वाले", "वुह", "वे", "वग़ैरह", "संग", "सकता", "सकती", "सकते",
    "सबसे", "सभि", "सभी", "समय", "समान", "साथ", "साबुत", "साभ", "सारा", "सारे", "से", "सो", "स्थान", "स्वयं", "हम", "हमने", "हमारा", "हि", "ही", "हुअ", "हुआ",
    "हुइ", "हुई", "हुए", "हुये", "हूँ", "हे", "हें", "है", "हैं", "हो", "होता", "होति", "होती", "होते", "होना", "होने", "के"
]

In [ ]:
# Ensure the column with text data is correctly specified (replace 'text_column' with the actual column name)
text_column = 'text'  # Replace with the correct column name containing text data

# Tokenize and preprocess the text data
tokens = [word_tokenize(doc.lower()) for doc in df[text_column].dropna()]  # Tokenize and convert to lowercase

# Flatten the list of tokens
all_tokens = [token for sublist in tokens for token in sublist]

# Define a function to check if a word is in Hindi
def is_hindi(word):
    # Check if the word contains Devanagari characters
    return bool(re.search(r'[\u0900-\u097F]', word))

# Get the vocabulary of only Hindi words
hindi_vocabulary = {token for token in all_tokens if is_hindi(token)}

print("Hindi Vocabulary:", hindi_vocabulary)
print("Hindi Vocabulary size:", len(hindi_vocabulary))

Hindi Vocabulary: {'गुंडे', 'रहेता', 'जाहिल', 'आरएसएस', 'भोलेनाथ', 'नज़रिया', 'दिसंबर', 'पाता', 'शुरू', 'जीडीपी', 'मजबूर', 'फितरत', 'शान', 'जन्मस्थान', 'धरोहर', 'लोकप्रियता।।।', 'बदतर', 'नफ़रत', 'बुन्ड', 'श्रद्धांजलियों', 'पसंद', 'शर्म', 'काबिलियत', 'गिटा', 'उन्ही', 'लेके', 'पड़', 'सम्मानित', 'अमित', 'वर्ग', 'महत्वपूर्ण', 'पाओगी', 'पत्तियां', 'ह्रीं', 'राष्ट्र', 'बुरा', 'मै', 'सदियों', 'लगाने', 'आयेगी', 'धर्मान्धता', 'हे।', 'जाओ', 'जी', 'प्रणाली', 'दिलवा', 'पोषण', 'सात', 'ग', 'आक्रांता', 'रख', 'कर', 'ताकी', 'व्यायाम', 'खातिर', 'रेहते', 'मिनिस्टर', '।।।', 'जिहादियों', 'है।वही', 'काटकर', 'अलग', 'पद्मिनी', 'अनपढ़ों', 'वहाँ', 'जिन्हे', 'कह', 'नदियाँ', 'लेती।', 'यूपी', 'अब', 'खंडहर', 'ऊंचा', 'तुझ', 'डाल', 'दृष्टि', 'बेवकूफ', 'निकाल', 'या', 'अधिकारों', 'बांटते', 'जाएगा😂😂', 'राष्ट्रपिता', 'डॉग', 'संवेदनशीलता', 'शाह', 'तुरंत', 'जिक्र', 'बुराई', 'शुक्ला', 'दे।', 'लग', 'जन्माष्टमी', 'देखिए', 'नर्क', 'मंत्रियों', 'श्रीराम🙏🚩', 'आवे', 'चंद', 'कन्याकुमारी', 'पुर्वजो', 'तुमको', 'बताकर', 'बंधुओ', 'आशा

In [ ]:
def remove_hindi_stopwords(text):
    words = word_tokenize(text.lower())  # Tokenize the text into words
    filtered_words = [word for word in words if word not in hindi_stopwords]  # Remove Hindi stopwords
    return ' '.join(filtered_words)

In [ ]:
df['cleaned_text'] = df['cleaned_text'].apply(lambda text: remove_hindi_stopwords(text) if pd.notnull(text) else text)

In [ ]:
output_file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/preprocessed_data.csv'

# Save the cleaned data to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Preprocessed data saved to {output_file_path}")

Preprocessed data saved to /content/drive/MyDrive/Internship_NIT_Patna/preprocessed_data.csv


In [ ]:
# Load the CSV file
file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/preprocessed_data.csv'
df = pd.read_csv(file_path)

# Function to tokenize text
def tokenize_text(text):
    return text.split(" ")

# Function to remove poorna virams from within tokens
def remove_purna_viram(tokens):
    return [token.replace('।', '') for token in tokens]

# Function to remove Urdu text
def remove_urdu_text(tokens):
    urdu_range = ('\u0600', '\u06FF')  # Unicode range for Arabic script
    return [token for token in tokens if not any(urdu_range[0] <= char <= urdu_range[1] for char in token)]

# Function to remove underscores
def remove_underscores(tokens):
    return [token.replace('_', '') for token in tokens]

# Function to remove empty tokens
def remove_empty_tokens(tokens):
    return [token for token in tokens if token]

# Function to check if a token contains both Hindi and non-Hindi characters
def contains_hindi_and_non_hindi(token):
    hindi_pattern = re.compile('[\u0900-\u097F]')
    non_hindi_pattern = re.compile('[^\u0900-\u097F\s]')
    contains_hindi = bool(hindi_pattern.search(token))
    contains_non_hindi = bool(non_hindi_pattern.search(token))
    return contains_hindi and contains_non_hindi

# Function to split tokens containing both Hindi and non-Hindi characters
def split_hindi_and_non_hindi(token):
    hindi_pattern = re.compile('([\u0900-\u097F]+)')
    non_hindi_pattern = re.compile('([^\u0900-\u097F]+)')
    split_tokens = hindi_pattern.split(token)
    split_tokens = [sub_token for token in split_tokens for sub_token in non_hindi_pattern.split(token) if sub_token]
    return split_tokens

# Apply the function to split the tokens and update the tokenized_text
def process_tokenized_text(tokenized_text):
    processed_tokens = []
    for token in tokenized_text:
        if contains_hindi_and_non_hindi(token):
            processed_tokens.extend(split_hindi_and_non_hindi(token))
        else:
            processed_tokens.append(token)
    return processed_tokens

# Function to process the dataframe
def tokenize_dataframe(df, text_column='cleaned_text'):
    df['tokenized_text'] = df[text_column].apply(lambda text: tokenize_text(text) if pd.notnull(text) else [])
    df['tokenized_text'] = df['tokenized_text'].apply(remove_purna_viram)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_urdu_text)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_underscores)
    df['tokenized_text'] = df['tokenized_text'].apply(remove_empty_tokens)
    df['tokenized_text'] = df['tokenized_text'].apply(process_tokenized_text)
    return df

# Process the dataframe
df = tokenize_dataframe(df, text_column='cleaned_text')

# Save the processed DataFrame to a CSV file
output_file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_cleaned_data.csv'
df.to_csv(output_file_path, index=False)

# Display the first few rows of the dataframe
print(df.head())

              tweet_id                                               text  \
0  1214794148504457217  It’s not a fight for dharma it’s embracing it....   
1  1242560043494621185  bhenchod mulk ki maa chud rahi hai aur tum log...   
2  1335539265015517186  #babrimasjid\n#BabriMasjid was and always will...   
3  1467692528308686850  #babrimasjid\n#BabriMasjid was and always will...   
4  1467694052644511745  #babrimasjid\n#BabriMasjid was and always will...   

  binary_label                                       cleaned_text  \
0          NOT       fight dharma embracing hindusagainsthindutva   
1          HOF  bhenchod mulk ki maa chud rahi hai aur tum log...   
2          NOT  babrimasjid babrimasjid always mosquerecently ...   
3          HOF  babrimasjid babrimasjid always mosquerecently ...   
4          HOF  babrimasjid babrimasjid always mosquerecently ...   

                                      tokenized_text  
0  [fight, dharma, embracing, hindusagainsthindutva]  
1  [bhenchod

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Internship_NIT_Patna/tokenized_cleaned_data.csv')

# Specify the column you want to delete
column_to_delete = 'text'

# Delete the column
df.drop(columns=['text'], inplace=True)

# Save the updated DataFrame back to the same CSV file, effectively overwriting it
df.to_csv('tokenized_cleaned_data.csv', index=False)

# Specify the column you want to delete
column_to_delete = 'cleaned_text'

# Delete the column
df.drop(columns=['cleaned_text'], inplace=True)

# Save the updated DataFrame back to the same CSV file, effectively overwriting it
df.to_csv('tokenized_text.csv', index=False)

In [ ]:
# from Preprocessed_and_tokenized import tokenize_dataframe

def load_and_process_data(file_path):
    """Load and process the data."""
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Process the dataframe
    df = tokenize_dataframe(df, text_column='cleaned_text')
    return df

def split_tokens_by_language(df, tokenized_text_column='tokenized_text'):
    """Split tokens into Hindi and English sets."""
    hindi_tokens = set()
    english_tokens = set()

    for tokens in df[tokenized_text_column]:
        for token in tokens:
            if re.search('[\u0900-\u097F]', token):  # Check for Hindi characters
                hindi_tokens.add(token)
            else:
                english_tokens.add(token)

    return hindi_tokens, english_tokens

def create_combined_vocabulary(hindi_tokens, english_tokens):
    """Create a combined vocabulary from Hindi and English tokens."""
    combined_vocabulary = hindi_tokens.union(english_tokens)
    return combined_vocabulary

def main(file_path):
    """Main function to load data, split tokens, and print vocabularies."""
    df = load_and_process_data(file_path)

    hindi_tokens, english_tokens = split_tokens_by_language(df, tokenized_text_column='tokenized_text')

    combined_vocabulary = create_combined_vocabulary(hindi_tokens, english_tokens)

    # Print the vocabularies to verify
    print("Hindi Vocabulary:")
    print(len(hindi_tokens))
    print(hindi_tokens)

    print("\nEnglish Vocabulary:")
    print(len(english_tokens))
    print(english_tokens)

    print("\nCombined Vocabulary:")
    print(len(combined_vocabulary))
    print(combined_vocabulary)

if __name__ == "__main__":
    file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_cleaned_data.csv'
    main(file_path)

Hindi Vocabulary:
4397
{'गुंडे', 'रहेता', 'जाहिल', 'आरएसएस', 'भोलेनाथ', 'नज़रिया', 'गंदेरीतिरिवाज', 'दिसंबर', 'पाता', 'शुरू', 'जीडीपी', 'मजबूर', 'फितरत', 'शान', 'जन्मस्थान', 'धरोहर', 'बदतर', 'नफ़रत', 'बुन्ड', 'श्रद्धांजलियों', 'पसंद', 'शर्म', 'काबिलियत', 'गिटा', 'बहुतबहुत', 'उन्ही', 'लेके', 'पड़', 'सम्मानित', 'अमित', 'पाओगी', 'महत्वपूर्ण', 'भोसदी', 'पत्तियां', 'ह्रीं', 'राष्ट्र', 'बुरा', 'सदियों', 'आयेगी', 'लगाने', 'धर्मान्धता', 'जाओ', 'जी', 'प्रणाली', 'दिलवा', 'पोषण', 'सात', 'ग', 'आक्रांता', 'रख', 'ताकी', 'व्यायाम', 'मिनिस्टर', 'खातिर', 'रेहते', 'कर', 'जिहादियों', 'काटकर', 'अलग', 'पद्मिनी', 'अनपढ़ों', 'जिन्हे', 'कह', 'नदियाँ', 'यूपी', 'खंडहर', 'ऊंचा', 'तुझ', 'डाल', 'दृष्टि', 'बेवकूफ', 'ब्रह्मांड', 'निकाल', 'गज्बेहै', 'अधिकारों', 'बांटते', 'राष्ट्रपिता', 'संवेदनशीलता', 'डॉग', 'शाह', 'तुरंत', 'जिक्र', 'बुराई', 'शुक्ला', 'लग', 'जन्माष्टमी', 'देखिए', 'नर्क', 'मंत्रियों', 'आवे', 'चंद', 'कन्याकुमारी', 'पुर्वजो', 'तुमको', 'बताकर', 'बंधुओ', 'आशाराम', 'होगी', 'खुलके', 'गिरती', 'बने', 'निम्न', 

In [ ]:
def split_tokens_by_language(df, tokenized_text_column='tokenized_text'):
    hindi_tokens = set()
    english_tokens = set()

    for tokens in df[tokenized_text_column]:
        for token in tokens:
            if re.search('[\u0900-\u097F]', token):  # Check for Hindi characters
                hindi_tokens.add(token)
            else:
                english_tokens.add(token)

    return hindi_tokens, english_tokens

def create_token_id_mapping(tokens):
    return {token: idx for idx, token in enumerate(tokens, start=1)}

def create_combined_token_id_mapping(hindi_tokens, english_tokens):
    combined_tokens = sorted(hindi_tokens.union(english_tokens))
    return {token: idx for idx, token in enumerate(combined_tokens, start=1)}


def unique_ids(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Process the dataframe
    df = tokenize_dataframe(df, text_column='cleaned_text')

    # Split tokens into Hindi and English sets
    hindi_tokens, english_tokens = split_tokens_by_language(df, tokenized_text_column='tokenized_text')

    # Create combined token to ID mapping
    combined_token_id_mapping = create_combined_token_id_mapping(hindi_tokens, english_tokens)

    return combined_token_id_mapping, df, hindi_tokens, english_tokens

def convert_tokens_to_ids(df, token_id_mapping):
    # Replace tokens with their corresponding IDs
    df['token_ID'] = df['tokenized_text'].apply(
        lambda tokens: [token_id_mapping.get(token, -1) for token in tokens]
    )
    return df

def separate_tokens_by_language(df, hindi_tokens, english_tokens, token_id_mapping):
    """Separate tokens into English and Hindi with their token IDs."""
    hindi_token_id_mapping = {token: token_id_mapping[token] for token in hindi_tokens if token in token_id_mapping}
    english_token_id_mapping = {token: token_id_mapping[token] for token in english_tokens if token in token_id_mapping}

    return hindi_token_id_mapping, english_token_id_mapping

if __name__ == "__main__":
    file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_cleaned_data.csv'

    # Get the token-ID mapping, DataFrame, and token sets
    combined_token_id_mapping, df, hindi_tokens, english_tokens = unique_ids(file_path)

    # Print the token-ID mapping
    print("\nCombined Token to ID Mapping:")
    print(combined_token_id_mapping)

    # Convert tokens to IDs
    df = convert_tokens_to_ids(df, combined_token_id_mapping)

    # Create a new DataFrame with only 'binary_label' and 'token_ID'
    output_df = df[['binary_label', 'token_ID']]

    # Save the new DataFrame to a CSV file
    output_file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_text_as_ids.csv'
    output_df.to_csv(output_file_path, index=False)

    print(f"\nNew CSV file with token IDs saved to: {output_file_path}")

    # Separate English and Hindi tokens with their token IDs
    hindi_token_id_mapping, english_token_id_mapping = separate_tokens_by_language(df, hindi_tokens, english_tokens, combined_token_id_mapping)

    # Print separated token-ID mappings
    print("\nHindi Token to ID Mapping:")
    print(hindi_token_id_mapping)

    print("\nEnglish Token to ID Mapping:")
    print(english_token_id_mapping)

    print(len(hindi_token_id_mapping))
    print(len(english_token_id_mapping))



Combined Token to ID Mapping:
{'aa': 1, 'aaba': 2, 'aabee': 3, 'aabhas': 4, 'aabhi': 5, 'aacha': 6, 'aadhe': 7, 'aadmi': 8, 'aadmiaadmi': 9, 'aaen': 10, 'aaenge': 11, 'aag': 12, 'aagaye': 13, 'aage': 14, 'aagey': 15, 'aagh': 16, 'aaha': 17, 'aai': 18, 'aaise': 19, 'aaj': 20, 'aakankshat': 21, 'aakarpatel': 22, 'aakhri': 23, 'aam': 24, 'aamal': 25, 'aameen': 26, 'aamircapri': 27, 'aamirkhan': 28, 'aamkhe': 29, 'aana': 30, 'aane': 31, 'aankhe': 32, 'aankhein': 33, 'aankho': 34, 'aao': 35, 'aaoge': 36, 'aap': 37, 'aapa': 38, 'aapas': 39, 'aaphonest': 40, 'aapka': 41, 'aapke': 42, 'aapki': 43, 'aapko': 44, 'aapna': 45, 'aapnareshbalyan': 46, 'aapne': 47, 'aapni': 48, 'aapse': 49, 'aarabi': 50, 'aaray': 51, 'aari': 52, 'aarti': 53, 'aartitva': 54, 'aaryaindian': 55, 'aas': 56, 'aashole': 57, 'aasiha': 58, 'aasmani': 59, 'aata': 60, 'aatanbadi': 61, 'aatankbaadi': 62, 'aatanki': 63, 'aatankwadii': 64, 'aatankwadiyo': 65, 'aate': 66, 'aati': 67, 'aatnkwad': 68, 'aatnkwadi': 69, 'aatnkwadiyon

Applying Tfidf Vectorizer on the text and getting results from Machine Learning Classifiers

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import ast

# Load the data
file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_cleaned_data.csv'
df = pd.read_csv(file_path)

# Check for NaN values in tokenized_text
print("Checking for empty entries in 'tokenized_text':")
print(df['tokenized_text'].isna().sum(), "NaN values found.")

# Convert the string representation of lists to actual lists
df['tokenized_text'] = df['tokenized_text'].apply(ast.literal_eval)

# Join the tokens back into strings for TF-IDF Vectorizer
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: ' '.join(x))

# Check if there are any empty entries after conversion
print(df[df['tokenized_text'] == ""])

# Create TF-IDF features
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['tokenized_text'])

# Convert the result to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, df['binary_label'], test_size=0.2, random_state=42)

# Logistic Regression Model
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)
y_logistic_pred = logistic_classifier.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_logistic_pred)
logistic_report = classification_report(y_test, y_logistic_pred, target_names=['NOT', 'HOF'])
print(f"Logistic Regression Accuracy: {logistic_accuracy * 100:.2f}%")
print("Logistic Regression Classification Report:")
print(logistic_report)
print("\n")

# SVM Model
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)
y_svm_pred = svm_classifier.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_svm_pred)
svm_conf_matrix = confusion_matrix(y_test, y_svm_pred)
svm_report = classification_report(y_test, y_svm_pred, target_names=['NOT', 'HOF'])
print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print("SVM Confusion Matrix:")
print(svm_conf_matrix)
print("SVM Classification Report:")
print(svm_report)
print("\n")

# Naive Bayes Model
naivebayes_classifier = GaussianNB()
naivebayes_classifier.fit(X_train, y_train)
y_naivebayes_pred = naivebayes_classifier.predict(X_test)
naivebayes_accuracy = accuracy_score(y_test, y_naivebayes_pred)
naivebayes_conf_matrix = confusion_matrix(y_test, y_naivebayes_pred)
naivebayes_report = classification_report(y_test, y_naivebayes_pred, target_names=['NOT', 'HOF'])
print(f"Naive Bayes Accuracy: {naivebayes_accuracy * 100:.2f}%")
print("Naive Bayes Confusion Matrix:")
print(naivebayes_conf_matrix)
print("Naive Bayes Classification Report:")
print(naivebayes_report)
print("\n")

# K-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier(n_neighbors=4)
knn_classifier.fit(X_train, y_train)
knn_predict = knn_classifier.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_predict)
print("K-Nearest Neighbors Classifier Results:")
print(f"Accuracy: {knn_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, knn_predict))
print("Classification Report:")
print(classification_report(y_test, knn_predict, target_names=['NOT', 'HOF']))
print("\n")

# Decision Tree Classifier
dectree_classifier = DecisionTreeClassifier()
dectree_classifier.fit(X_train, y_train)
dectree_predict = dectree_classifier.predict(X_test)
dectree_accuracy = accuracy_score(y_test, dectree_predict)
print("Decision Tree Classifier Results:")
print(f"Accuracy: {dectree_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, dectree_predict))
print("Classification Report:")
print(classification_report(y_test, dectree_predict, target_names=['NOT', 'HOF']))
print("\n")

# Gradient Boosting Classifier
gradboost_classifier = GradientBoostingClassifier(random_state=0)
gradboost_classifier.fit(X_train, y_train)
gradboost_predict = gradboost_classifier.predict(X_test)
gradboost_accuracy = accuracy_score(y_test, gradboost_predict)
print("Gradient Boosting Classifier Results:")
print(f"Accuracy: {gradboost_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, gradboost_predict))
print("Classification Report:")
print(classification_report(y_test, gradboost_predict, target_names=['NOT', 'HOF']))
print("\n")

# Random Forest Classifier
forest_classifier = RandomForestClassifier()
forest_classifier.fit(X_train, y_train)
forest_predict = forest_classifier.predict(X_test)
forest_accuracy = accuracy_score(y_test, forest_predict)
print("Random Forest Classifier Results:")
print(f"Accuracy: {forest_accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(y_test, forest_predict))
print("Classification Report:")
print(classification_report(y_test, forest_predict, target_names=['NOT', 'HOF']))
print("\n")


Checking for empty entries in 'tokenized_text':
0 NaN values found.
Empty DataFrame
Columns: [tweet_id, text, binary_label, cleaned_text, tokenized_text]
Index: []
Logistic Regression Accuracy: 73.75%
Logistic Regression Classification Report:
              precision    recall  f1-score   support

         NOT       0.76      0.72      0.74       515
         HOF       0.71      0.76      0.73       468

    accuracy                           0.74       983
   macro avg       0.74      0.74      0.74       983
weighted avg       0.74      0.74      0.74       983



SVM Accuracy: 73.96%
SVM Confusion Matrix:
[[361 154]
 [102 366]]
SVM Classification Report:
              precision    recall  f1-score   support

         NOT       0.78      0.70      0.74       515
         HOF       0.70      0.78      0.74       468

    accuracy                           0.74       983
   macro avg       0.74      0.74      0.74       983
weighted avg       0.74      0.74      0.74       983



Naive

In [ ]:
def calculate_percentage_in_ranges(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Convert the 'token_ID' column from string representation of lists to actual lists
    df['token_ID'] = df['token_ID'].apply(eval)

    # Count the number of tokens in each row
    df['token_ID_length'] = df['token_ID'].apply(len)

    min_length = df['token_ID_length'].min()
    max_length = df['token_ID_length'].max()
    average_length = df['token_ID_length'].mean()

    print(f"Minimum length of 'token_ID': {min_length}")
    print(f"Maximum length of 'token_ID': {max_length}")
    print(f"Average length of 'token_ID': {average_length:.2f}")

    # Define the ranges
    ranges = {
        "10-20": (10, 20),
        "20-30": (20, 30),
        "30-40": (30, 40),
        "40-50": (40, 50),
        "50-60": (50, 60),
        "60-70": (60, 70),
        "70-80": (70, 80),
        "80-90": (80, 90),
        "90-100": (90, 100)
    }

    # Initialize a dictionary to hold counts for each range
    counts = {range_label: 0 for range_label in ranges}

    # Total number of rows
    total_rows = len(df)

    # Count the number of lengths in each range
    for range_label, (start, end) in ranges.items():
        counts[range_label] = df['token_ID_length'].between(start, end).sum()

    # Calculate and print percentages for each range
    for range_label, count in counts.items():
        percentage = (count / total_rows) * 100
        print(f"Percentage of lengths in range {range_label}: {percentage:.2f}%")

if __name__ == "__main__":
    # File path
    file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_text_as_ids.csv'

    # Calculate and print percentages in each range
    calculate_percentage_in_ranges(file_path)


Minimum length of 'token_ID': 3
Maximum length of 'token_ID': 107
Average length of 'token_ID': 38.16
Percentage of lengths in range 10-20: 11.72%
Percentage of lengths in range 20-30: 22.53%
Percentage of lengths in range 30-40: 26.39%
Percentage of lengths in range 40-50: 20.72%
Percentage of lengths in range 50-60: 15.28%
Percentage of lengths in range 60-70: 6.17%
Percentage of lengths in range 70-80: 2.81%
Percentage of lengths in range 80-90: 0.96%
Percentage of lengths in range 90-100: 0.33%


In [ ]:
def pad_or_truncate_sequences(sequences, max_length):
    """Pads or truncates sequences to ensure a consistent length."""
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            padded_sequences.append(seq[:max_length])
        else:
            padded_sequences.append(seq + [0] * (max_length - len(seq)))  # Pad with zeros
    return padded_sequences

def process_sequences(file_path, output_path, max_length=50):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Ensure token_ID column is a list of integers
    df['token_ID'] = df['token_ID'].apply(eval).apply(lambda x: list(map(int, x)))

    # Pad or truncate sequences
    df['token_ID'] = pad_or_truncate_sequences(df['token_ID'], max_length)

    # Print the first 20 rows of the padded sequences
    print("First 20 rows of padded token_ID sequences:")
    print(df[['token_ID']].head(20))

    # Save the processed DataFrame to a new CSV file
    df.to_csv(output_path, index=False)
    print(f"Processed CSV file with padded/truncated sequences saved to: {output_path}")

if __name__ == "__main__":
    input_file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_text_as_ids.csv'
    output_file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokenized_text_as_padded_ids.csv'

    process_sequences(input_file_path, output_file_path, max_length=50)


First 20 rows of padded token_ID sequences:
                                             token_ID
0   [3970, 3174, 3611, 4929, 0, 0, 0, 0, 0, 0, 0, ...
1   [1538, 7671, 6324, 6970, 2327, 9246, 4639, 949...
2   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
3   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
4   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
5   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
6   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
7   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
8   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
9   [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
10  [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
11  [1059, 1059, 495, 7582, 4625, 1026, 7583, 4502...
12  [9931, 438, 6252, 6085, 6067, 6252, 7822, 1018...
13  [9931, 438, 6252, 6085, 6067, 6252, 7822, 1018...
14  [9931, 438, 6252, 6085, 6067, 6252, 7822, 1018...
15  [9931, 438, 6252, 6085, 6067, 6252, 7822, 1018...
16  [9931, 438, 6252, 6085, 6067, 6252

In [ ]:
# Load the CSV file
data = pd.read_csv('/content/drive/MyDrive/Internship_NIT_Patna/tokenized_text_as_padded_ids.csv')

# Ensure 'token_ID' column is in list format if it was saved as strings
data['token_ID'] = data['token_ID'].apply(eval).apply(lambda x: list(map(int, x)))

# Calculate the length of 'token_ID' for the first 10 rows
lengths = data['token_ID'].head(4000).apply(len)

# Print the lengths
print("Lengths of the first 10 rows of 'token_ID':")
print(lengths.tolist())


Lengths of the first 10 rows of 'token_ID':
[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50,

In [ ]:
file_path = r'/content/drive/MyDrive/Internship_NIT_Patna/tokens_with_ids.txt'
# /content/tokens_with_ids.txt
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Print the first few lines to understand the format
for i, line in enumerate(lines[:10]):
    print(f"Line {i}: {line.strip()}")

Line 0: Hindi Token to ID Mapping:
Line 1: {'वनः': 16764, 'बीबी': 15917, 'शोध': 16981, 'क़दम': 13852, 'अलमुंतशिर': 13337, 'चरवाहे': 14363, 'सत्ता': 17071, 'मठाधीशों': 16165, 'सुनना': 17281, 'फैलाई': 15716, 'सुरमा': 17297, 'चारदीवारी': 14396, 'गोबरसेवन': 14287, 'हारते': 17442, 'अपूर्णीय': 13293, 'खंजर': 14061, 'सुंदर': 17265, 'महासचिव': 16232, 'गृहमंत्री': 14275, 'सिंगल': 17223, 'भाग': 16067, 'घातक': 14313, 'ओवैसी': 13693, 'हकदार': 17377, 'नमस्कार': 15200, 'भूखे': 16098, 'पड़े': 15355, 'रिण': 16579, 'चीखना': 14422, 'फैलाते': 15719, 'बेचारी': 15940, 'सिस्टम': 17248, 'चलेगी': 14378, 'देवता': 15097, 'हैवैसे': 17566, 'ट्वीटर': 14788, 'महासभा': 16233, 'वैमनस्यता': 16885, 'बेड': 15949, 'सवर्ण': 17165, 'फैलायेंगे': 15722, 'उनपे': 13621, 'अफसोस': 13299, 'चौक': 14472, 'बदज़ुबान': 15786, 'लगेंगे': 16643, 'गर': 14193, 'छाप': 14487, 'छिप': 14488, 'अत्याचार': 13245, 'टारगेट': 14753, 'इमारतों': 13547, 'बेशर्म': 15959, 'खलिसतनी': 14077, 'देखें': 15082, 'हालत': 17446, 'तोबौद्धजैनसिख': 14937, 'अली': 133

In [ ]:
# Extract Hindi and English dictionaries from the file
hindi_dict = {}
english_dict = {}

# Identify sections and parse the dictionaries
section = None
for line in lines:
    line = line.strip()
    if line.startswith('Hindi Token to ID Mapping:'):
        section = 'hindi'
    elif line.startswith('English Token to ID Mapping:'):
        section = 'english'
    elif section == 'hindi':
        try:
            hindi_dict.update(json.loads(line.replace("'", '"')))
        except json.JSONDecodeError:
            print(f"Failed to parse line as JSON: {line}")
    elif section == 'english':
        try:
            english_dict.update(json.loads(line.replace("'", '"')))
        except json.JSONDecodeError:
            print(f"Failed to parse line as JSON: {line}")

# Print the first few entries to verify
print("Hindi Dictionary Sample:", list(hindi_dict.items())[:5])
print("English Dictionary Sample:", list(english_dict.items())[:5])


Failed to parse line as JSON: 
Hindi Dictionary Sample: [('वनः', 16764), ('बीबी', 15917), ('शोध', 16981), ('क़दम', 13852), ('अलमुंतशिर', 13337)]
English Dictionary Sample: [('release', 9870), ('missionaries', 7739), ('mature', 7543), ('chant', 2234), ('pradeepgupta', 9223)]


In [ ]:
# Initialize lists to store Hindi and English tokens
hindi_tokens = []
english_tokens = []

# Define regex patterns for Hindi and English tokens
hindi_pattern = re.compile(r'[\u0900-\u097F]+')
english_pattern = re.compile(r'[a-zA-Z]+')

# Separate tokens based on language
for token, unique_id in hindi_dict.items():
    if hindi_pattern.search(token):
        hindi_tokens.append((token, unique_id))

for token, unique_id in english_dict.items():
    if english_pattern.search(token):
        english_tokens.append((token, unique_id))

# Print the results (optional)
print("Hindi Tokens:", hindi_tokens)
print("English Tokens:", english_tokens)

Hindi Tokens: [('वनः', 16764), ('बीबी', 15917), ('शोध', 16981), ('क़दम', 13852), ('अलमुंतशिर', 13337), ('चरवाहे', 14363), ('सत्ता', 17071), ('मठाधीशों', 16165), ('सुनना', 17281), ('फैलाई', 15716), ('सुरमा', 17297), ('चारदीवारी', 14396), ('गोबरसेवन', 14287), ('हारते', 17442), ('अपूर्णीय', 13293), ('खंजर', 14061), ('सुंदर', 17265), ('महासचिव', 16232), ('गृहमंत्री', 14275), ('सिंगल', 17223), ('भाग', 16067), ('घातक', 14313), ('ओवैसी', 13693), ('हकदार', 17377), ('नमस्कार', 15200), ('भूखे', 16098), ('पड़े', 15355), ('रिण', 16579), ('चीखना', 14422), ('फैलाते', 15719), ('बेचारी', 15940), ('सिस्टम', 17248), ('चलेगी', 14378), ('देवता', 15097), ('हैवैसे', 17566), ('ट्वीटर', 14788), ('महासभा', 16233), ('वैमनस्यता', 16885), ('बेड', 15949), ('सवर्ण', 17165), ('फैलायेंगे', 15722), ('उनपे', 13621), ('अफसोस', 13299), ('चौक', 14472), ('बदज़ुबान', 15786), ('लगेंगे', 16643), ('गर', 14193), ('छाप', 14487), ('छिप', 14488), ('अत्याचार', 13245), ('टारगेट', 14753), ('इमारतों', 13547), ('बेशर्म', 15959), ('खलिस

In [ ]:
# Create embeddings for Hindi tokens using FastText
# Load pre-trained FastText model for Hindi
fasttext_model = fasttext.load_model('/content/drive/MyDrive/Internship_NIT_Patna/cc.hi.300.bin')

# Create a dictionary to store Hindi embeddings
hindi_embeddings = {}
for token, unique_id in hindi_tokens:
    embedding = fasttext_model.get_word_vector(token)
    hindi_embeddings[unique_id] = embedding

# Print Hindi embeddings (optional)
print("Hindi Embeddings:", hindi_embeddings)

Streaming output truncated to the last 5000 lines.
        6.27788063e-03, -2.84734629e-02,  5.73073551e-02, -5.94868474e-02,
       -3.92655134e-02, -2.32095346e-02,  9.18731689e-02, -3.39465193e-03,
        1.27708480e-01, -4.14486229e-02,  2.78731212e-02, -1.94838699e-02,
       -4.62052524e-02, -5.75155541e-02,  7.83847496e-02, -7.06831142e-02,
       -4.24088817e-03,  1.24933273e-01,  2.87839510e-02, -3.12993675e-02,
       -6.20394163e-02, -7.23190829e-02,  2.08035093e-02, -1.23461597e-02,
        6.62924945e-02,  2.35770904e-02, -1.51522066e-02,  2.65818238e-02,
        5.96136451e-02,  3.47618312e-02,  7.66692217e-03, -4.58976440e-02,
       -2.98190601e-02,  8.23642612e-02,  5.56237996e-02, -1.28258520e-03,
       -1.90429948e-02, -6.22461252e-02, -5.89125305e-02,  3.46049592e-02,
        1.08609572e-02,  1.77339055e-02, -1.69689432e-02,  9.91642103e-02,
       -2.01770514e-02, -5.74389696e-02,  5.27150445e-02,  3.77841890e-02,
        7.52676353e-02, -6.29159957e-02,  1.97507

In [ ]:
glove_file = '/content/drive/MyDrive/Internship_NIT_Patna/glove.6B.300d.txt'
# glove_file = '/content/drive/MyDrive/Internship_NIT_Patna/cc.en.300.bin'
glove_model = {}
with open(glove_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_model[word] = vector

# Create a dictionary to store English embeddings
english_embeddings = {}
count=0
for token, unique_id in english_tokens:
    embedding = glove_model.get(token)
    if embedding is not None:
        english_embeddings[unique_id] = embedding
    else:
        print(f"Token '{token}' not found in GloVe model.")
        count += 1

# Print English embeddings (optional)
print("English Embeddings:", english_embeddings)






# import fasttext
# import fasttext.util
# import numpy as np

# Load FastText model
# fasttext_model = fasttext.load_model('/content/drive/MyDrive/Internship_NIT_Patna/cc.en.300.bin')

# # Create a dictionary to store English embeddings
# english_embeddings = {}
# count = 0

# for token, unique_id in english_tokens:
#     embedding = fasttext_model.get_word_vector(token)
#     if embedding is not None and np.any(embedding):  # Check if embedding is not an array of zeros
#         english_embeddings[unique_id] = embedding
#     else:
#         print(f"Token '{token}' not found in FastText model.")
#         count += 1

# # Print English embeddings (optional)
# print("English Embeddings:", english_embeddings)


In [ ]:
english_embeddings

In [ ]:
len(hindi_embeddings)

In [ ]:
len(english_embeddings)

In [ ]:
print(count)